In [1]:
print("vi")

vi


In [2]:
import os
import psycopg2
from psycopg2 import sql

# Set the PGDATABASE environment variable
os.environ["PGDATABASE"] = "ernieplus"

def insert_values_into_table(values_list):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect("")
        cur = conn.cursor()

        
        args = ','.join(cur.mogrify("(%s,%s,%s, %s, %s, %s, %s, %s, %s, %s)", i).decode('utf-8')
                for i in values_list)
        
        cur.execute("INSERT INTO hm31.pubmed_all_xmls_abstracts VALUES " + (args))

        conn.commit()
        print("Inserted values successfully")

    except Exception as e:
        print("Error:", e)

    finally:
        cur.close()
        conn.close()

# Example usage:
values_to_insert = [
    (123425, '', '', '', 0, 0, '', 'doi1','title1', 'abs1'),
    (543251, '5678-1234', 'Grant3, Grant4', 'Chemical3, Chemical4', 2021, 2020, 'Term3, Term4','doi2','t1','a1')
]

insert_values_into_table(values_to_insert)


Error: relation "hm31.pubmed_all_xmls_abstracts" does not exist
LINE 1: INSERT INTO hm31.pubmed_all_xmls_abstracts VALUES (123425,''...
                    ^


In [3]:
import xmltodict
import pandas as pd

def parse(file_path):
    try:
        # Open and read the XML file
        with open(file_path, 'r', encoding='utf-8') as file:
            xml_contents = file.read()
        print("read")
        # Parse the XML content using xmltodict
        xml_dict = xmltodict.parse(xml_contents)
        print("parsed")
        # Print the parsed XML as a Python dictionary
    except Exception as e:
        print("Error:", e)
        
    return xml_dict

xml_dict = parse('/shared/hossein_hm31/xml_data/pubmed23n0001.xml')



read
parsed


In [4]:
import metapub

!convert pmid2doi 10



ModuleNotFoundError: No module named 'metapub'

In [ ]:
from dicttoxml import dicttoxml
xml = dicttoxml(xml_dict['PubmedArticleSet']['PubmedArticle'][0])
print(type(xml))
# print(str(xml))

In [ ]:
print(len(xml_dict['PubmedArticleSet']['PubmedArticle']))
# print(xml_dict['PubmedArticleSet']['PubmedArticle'][0])
print(type(xml_dict['PubmedArticleSet']['PubmedArticle'][0]))


In [ ]:
article_id_list = temp['PubmedData']['ArticleIdList']['ArticleId']
print('doi' in str(article_id_list[1]))
# print(article_id_list[1])
print(article_id_list[1].keys())
print(article_id_list[1]['#text'])


# Iterate through the list of ArticleId entries
# for article_id in article_id_list['ArticleId']:
#     if article_id.get('@IdType') == 'doi':
#         doi = article_id['#text']
#         print("DOI:", doi)
#         break

In [ ]:


text = """
<Day>16</Day>
</PubMedPubDate>
<PubMedPubDate PubStatus="medline">
<Year>1976</Year>
<Month>3</Month>
<Day>16</Day>
<Hour>0</Hour>
<Minute>1</Minute>
</PubMedPubDate>
<PubMedPubDate PubStatus="entrez">
<Year>1976</Year>
<Month>3</Month>
<Day>16</Day>
<Hour>0</Hour>
<Minute>0</Minute>
</PubMedPubDate>
</History>
<PublicationStatus>ppublish</PublicationStatus>
<ArticleIdList>
<ArticleId IdType="pubmed">4311</ArticleId>
<ArticleId IdType="doi">10.1111/j.1432-1033.1976.tb10218.x</ArticleId>
</ArticleIdList>
</PubmedData>
</PubmedArticle>
</PubmedArticleSet>
"""

doi_pattern = r'\b10\.\d{4,}/\S+'
doi_pattern = r'\b10\.\d{4,}(?:\.\d+)*\/[^\s]+'


# Find all DOIs in the text
dois = re.findall(doi_pattern, text)

# Print the found DOIs
for doi in dois:
    print("Found DOI:", doi)

In [ ]:
import re
def find_doi(string):
    doi_pattern = r'\b10\.\d{4,}(?:\.\d+)*\/[^\s]+'
    dois = re.findall(doi_pattern, string)
    return dois

    

In [ ]:
def clean_abstract(abstract):
    overhauled = ''
    
    if type(abstract) != str:    
      for a in abstract:
        if type(a) != str:
            if a != None and '#text' in a:
                overhauled += a['#text']
 
        else:
            overhauled += a
        
      return overhauled

    return abstract 

In [ ]:
from tqdm import tqdm

def parse_single_record(xml_dict, originial_dict, i=0):
    mesh_headings = []
    grants = []
    year = ""
    journal_ISSN = ""
    chemical_list = []
    meta_data = {}
    PMID = ""
    doi = ""
    title = ""
    abstract = ""

    year_revised = ""
    month_revised = ""
    day_revised = ""
    date_revised = ""
    
    journal_puddate = ""

    try:
        xml_dict = dict(xml_dict)

        #         print(xml_dict)

        try:
            if 'PMID' in xml_dict:
                PMID = xml_dict['PMID']['#text']
        except:
            pass

        try:
            if 'DateRevised' in xml_dict:

                new_dic = dict(xml_dict['DateRevised'])


                if 'Year' in new_dic:
                    year_revised = new_dic['Year']

                if 'Month' in new_dic:
                    month_revised = new_dic['Month']

                if 'Day' in new_dic:
                    day_revised = new_dic['Day']

        except:
            pass
       

        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])
                journal_ISSN = new_dic['Journal']['ISSN']['#text']

        except:
            pass

        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])
                new_dic = str(new_dic['Journal']['JournalIssue']['PubDate'])
                
                pattern = re.compile(r'\d{4}')
                year = pattern.findall(new_dic)[0]

   

        except:
            pass

        try:

            article_id_list = originial_dict['PubmedData']['ArticleIdList']['ArticleId']

            for element in article_id_list:
                if 'doi' in str(element).lower():
                    doi = element['#text']
                    break
        #                 print('doi' in str(article_id_list[1]))
        #                 # print(article_id_list[1])
        #                 print(article_id_list[1].keys())
        #                 print(article_id_list[1]['#text'])
        #                 #print('doi' in str(article_id_list[1]))

        except:
            pass

        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])

            if 'Title' in new_dic['Journal'].keys():
                journal_title = new_dic['Journal']['Title']

        except:
            pass

        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])

                title = new_dic['ArticleTitle']

                #                 if i == 10844:
                #                     print(title,'AAA \n')

                if '#text' in title:
                    title = title['#text']

                #                     if i == 10844:
                #                         print(title,'BBB \n')

                elif 'b' in title:
                    title = title['b']
                    if '#text' in title:
                        title = title['#text']


                elif 'sup' in title:
                    title = title['sup']

                if 'i' in title:
                    title = title['i']

                if type(title) == list:
                    title = ' '.join(title)




        except:
            pass

        try:

            if 'Article' in xml_dict: 
                new_dic = dict(xml_dict['Article'])
                abstract = new_dic['Abstract']['AbstractText']
                
                while '#text' in abstract:
                    abstract = abstract['#text']
                                        


        except:
            pass

        try:
            if 'Article' in xml_dict:
                new_dic = dict(xml_dict['Article'])

            if 'GrantList' in new_dic:
                if type(new_dic['GrantList']['Grant']) == list:
                    for grant in new_dic['GrantList']['Grant']:
                        if 'GrantID' in grant:
                            grants.append((grant['GrantID']))

                else:
                    grants.append(new_dic['GrantList']['Grant']['GrantID'])
                    pass
        except:
            pass

        try:
            if 'MeshHeadingList' in xml_dict:
                if type(xml_dict['MeshHeadingList']['MeshHeading']) == list:
                    for mesh in xml_dict['MeshHeadingList']['MeshHeading']:
                        if '@Type' in mesh['DescriptorName'].keys() and mesh['DescriptorName']['@Type'] == 'Geographic':
                            continue

                        mesh_headings.append((mesh['DescriptorName']['@UI']))

                else:

                    mesh = xml_dict['MeshHeadingList']['MeshHeading']['DescriptorName']
                    if (not '@Type' in mesh.keys() or mesh['@Type'] != 'Geographic'):
                        mesh_headings.append(xml_dict['MeshHeadingList']['MeshHeading']['DescriptorName']['@UI'])

        except:
            pass

        try:

            if 'ChemicalList' in xml_dict:
                if type(xml_dict['ChemicalList']['Chemical']) == list:
                    for substance in xml_dict['ChemicalList']['Chemical']:
                        if substance['NameOfSubstance']['@UI'][0].lower() == 'c':
                            chemical_list.append(substance['NameOfSubstance']['@UI'])

                else:

                    if xml_dict['ChemicalList']['Chemical']['NameOfSubstance']['@UI'][0].lower() == 'c':
                        chemical_list.append(xml_dict['ChemicalList']['Chemical']['NameOfSubstance']['@UI'])


        except Exception as e:
            print(e)
            print("ERR")
            pass

    except:
        pass


    if title == None:
        title = ''

    if abstract == None:
        abstract = ''

    abstract = clean_abstract(abstract)

    if year_revised == '':
        date_revised = '0000-00-00'

    else:
        date_revised = f'{year_revised}-{month_revised}-{day_revised}'
        # print(date_revised)

    meta_data = {'PMID': int(PMID), 'mesh': str(mesh_headings), 'grants': str(grants), 'year': str(year),
                 'journal_ISSN': str(journal_ISSN), 'journal_title': str(journal_title),
                 'chemical': str(chemical_list), 'doi': doi.lower(),
                 'title': str(title), 'abstract': str(abstract),
                 'date_revised': date_revised}

    return meta_data

meta_data_array = []

counter = 0
for i in tqdm(range(len(xml_dict['PubmedArticleSet']['PubmedArticle']))):
    
    rec = xml_dict['PubmedArticleSet']['PubmedArticle'][i]['MedlineCitation']
    total = xml_dict['PubmedArticleSet']['PubmedArticle'][i]
    x=parse_single_record(rec, total,i)
    meta_data_array.append(x)
    
    if x['abstract'] == '':
        counter += 1
    
    
    if type(x['title']) != str:
        print(x['title'])
        print(type(x['title']))
        print(i)

print(counter)
    
    

<h2>Saving into parquet </h2>
<p> We will convert the metadata array to dataframe, write it to parquet and read it and inspect the saved file against the original array as a cross-check mechanism</p>

In [ ]:
df = pd.DataFrame(meta_data_array)
df.set_index(['PMID', 'doi'], inplace=True)
df.head()
df.to_parquet('myfile.parquet')
print(df.head(2))
!rm myfile.parquet

In [ ]:
read_df = pd.read_parquet('myfile.parquet')
print(read_df.head(2))

<h2>As we see here, very few dois is available in the XMLS. This is cross checked with looking for 'doi' occurence in the string</h2>

In [ ]:
def investigate_for_doi_manually(xml_dict):
    xml = str(dicttoxml(xml_dict))
    return 'doi' in xml
 
total_doi = 0
for idx, meta in tqdm(enumerate(meta_data_array),  total = len(meta_data_array)):
    if meta['doi'] != '':
        total_doi += 1

print(f'totaly {total_doi} dois available from {len(meta_data_array)}')

In [ ]:
# total_doi = 0
for idx, meta in tqdm(enumerate(meta_data_array),  total = len(meta_data_array)):
    if meta['doi'] != '':
        total_doi += 1
    
    else:
        exist = investigate_for_doi_manually( xml_dict['PubmedArticleSet']['PubmedArticle'][i])
        if exist:
            print(meta['PMID'])
            break


In [ ]:
#Convert dict to query
def convert_dict_to_query(dic):
    if dic['year'] == '':
        year = 0
    
    else:
        year = dic['year']
        
    
    if dic['pub_year'] == '':
        pub_year = 0
    
    else:
        pub_year = dic['pub_year']
        
        
    query = (int(dic['PMID']), dic['journal_ISSN'], ' '.join(dic['grants']), ' '.join(dic['chemical']), 
            pub_year, year, ' '.join(dic['mesh']), dic['doi'], dic['abstract'], dic['title'] )
        
    return query

In [ ]:
query_array = []
na = 0
nt = 0
for i in meta_data_array:
    query_array.append(convert_dict_to_query(i))
    
#     print(i['title'])
    if len (i['title']) > 0:
        na +=1 
    
    if len (i['abstract']) > 0:
        nt += 1

print(na)
print(nt)

for query in query_array:
    if type(query[-1]) != str:
        print(query[-1])

# insert_values_into_table(query_array)

In [ ]:
def parallelize(file_path, mode='parquet', dump_address = '/shared/hossein_hm31/date_parquet/' ):
    xml_dict = parse(file_path)
    
    meta_data_array = []

    for i in range(len(xml_dict['PubmedArticleSet']['PubmedArticle'])):
        rec = xml_dict['PubmedArticleSet']['PubmedArticle'][i]['MedlineCitation']
        total = xml_dict['PubmedArticleSet']['PubmedArticle'][i]
        x=parse_single_record(rec, total)
        meta_data_array.append(x)
        
    if mode == 'postgres':
        query_array = []

        for i in meta_data_array:
            query_array.append(convert_dict_to_query(i))


        insert_values_into_table(query_array)
        
    elif mode == 'parquet':
        try:
            xml_name = file_path.split('/')[-1].split('.')[0] #Example '/shared/hossein_hm31/xml_data/pubmed23n0933.xml' -> pubmed23n0933.xml
            df = pd.DataFrame(meta_data_array)
            df.set_index(['PMID', 'doi'], inplace=True)
            df.to_parquet(f'{dump_address+xml_name}.parquet')
            print('saved')
        except Exception as e: # work on python 3.x
            print(str(e)+'\n')


#23 Mb vs 230 Mb single file

In [ ]:
import multiprocessing
import time

start = time.time()

nest_dir = '/shared/hossein_hm31/xml_data/'
files = os.listdir(nest_dir)

lst = []

for file in files:
    lst.append((nest_dir+file,))
    
with multiprocessing.Pool(processes=80) as pool:
    results = pool.starmap(parallelize, lst)
    
end = time.time()
print(f'elapsed {end - start}')


<h2> Finishing the unfinished ones </h2>
<p> Get the incomplete files </p>

In [ ]:
import multiprocessing
import time
import os

obtained_files = os.listdir('/shared/hossein_hm31/date_parquet/')
all_files = os.listdir('/shared/hossein_hm31/xml_data/')

all_xml_files = [file.split('.')[0] for file in all_files if '.xml' in file]
all_parquet_files = [file.split('.')[0] for file in obtained_files]

files_left = [file for file in all_xml_files if not file in all_parquet_files]
print(files_left)
print(len(files_left))

lst = []
start = time.time()


for file in files_left:
    lst.append((f'/shared/hossein_hm31/xml_data/{file}.xml',))


with multiprocessing.Pool(processes=60) as pool:
    results = pool.starmap(parallelize, lst)
    
end = time.time()
print(f'elapsed {end - start}')
